# Regression Model A: Field 4

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 4 (F4)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E4" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E4" ),]
dE4=d4[which(d4$Estate=="E4" ),]
dE5=d5[which(d5$Estate=="E4" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Regression

Fit the data using multiple regression for all variables. 

In [7]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=dg1)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-198.16  -50.42  -10.08   44.68  207.94 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   342.686813  72.424674   4.732  3.6e-05 ***
rainfall        0.004193   0.040006   0.105   0.9171    
foliar         -9.554472   4.674578  -2.044   0.0485 *  
prun.time      -0.851973  11.746912  -0.073   0.9426    
prun.cost       0.007246   0.006800   1.065   0.2940    
manuring.time -18.965047  30.906633  -0.614   0.5434    
manuring.cost  -0.001625   0.001215  -1.338   0.1897    
pd.time        26.565735  21.737805   1.222   0.2298    
pd.cost        -0.002739   0.006576  -0.417   0.6796    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 91.13 on 35 degrees of freedom
Multiple R-squared:  0.2863,	Adjusted R-squared:  0.1231 

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [8]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,193.1128,193.1128,0.02325263,0.87967764
foliar,1,49570.2891,49570.2891,5.96873585,0.01974825
prun.time,1,6336.9863,6336.9863,0.76303362,0.38833496
prun.cost,1,2092.6483,2092.6483,0.25197483,0.61883135
manuring.time,1,26281.2207,26281.2207,3.16450977,0.08394201
manuring.cost,1,18943.7315,18943.7315,2.28100605,0.13994319
pd.time,1,11718.1199,11718.1199,1.41097346,0.24289349
pd.cost,1,1440.6910,1440.6910,0.17347295,0.67958681
Residuals,35,290674.6354,8304.9896,NA,NA


### Using stepwise

In [9]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=405.01
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- prun.time      1        44 290718 403.02
- rainfall       1        91 290766 403.03
- pd.cost        1      1441 292115 403.23
- manuring.time  1      3127 293802 403.48
- prun.cost      1      9428 300103 404.42
- pd.time        1     12404 303078 404.85
<none>                       290675 405.01
- manuring.cost  1     14859 305533 405.21
- foliar         1     34695 325370 407.98

Step:  AIC=403.02
yield ~ rainfall + foliar + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq    RSS    AIC
- rainfall       1       122 290840 401.04
- pd.cost        1      1406 292124 401.23
- manuring.time  1      3284 294003 401.51
- pd.time        1     12595 303313 402.89
<none>                       290718 403.02
- manuring.cost  1     14874 305593 403.22
- prun.cost      1     1641


Call:
lm(formula = yield ~ foliar + prun.cost + manuring.cost, data = dg1)

Residuals:
     Min       1Q   Median       3Q      Max 
-191.077  -49.866    1.212   30.546  219.590 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    4.071e+02  2.591e+01  15.713   <2e-16 ***
foliar        -1.043e+01  4.010e+00  -2.600   0.0130 *  
prun.cost      6.541e-03  4.537e-03   1.442   0.1572    
manuring.cost -2.214e-03  9.357e-04  -2.367   0.0229 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 87.62 on 40 degrees of freedom
Multiple R-squared:  0.246,	Adjusted R-squared:  0.1895 
F-statistic:  4.35 on 3 and 40 DF,  p-value: 0.00961


       foliar     prun.cost manuring.cost 
     1.041620      1.082746      1.046860 



Call:
lm(formula = yield ~ foliar + prun.cost + manuring.cost, data = dg1)

Residuals:
     Min       1Q   Median       3Q      Max 
-191.077  -49.866    1.212   30.546  219.590 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    4.071e+02  2.591e+01  15.713   <2e-16 ***
foliar        -1.043e+01  4.010e+00  -2.600   0.0130 *  
prun.cost      6.541e-03  4.537e-03   1.442   0.1572    
manuring.cost -2.214e-03  9.357e-04  -2.367   0.0229 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 87.62 on 40 degrees of freedom
Multiple R-squared:  0.246,	Adjusted R-squared:  0.1895 
F-statistic:  4.35 on 3 and 40 DF,  p-value: 0.00961


## Prediction

Predict with the resample values.

In [10]:
set.seed(123)
dg3=sample(1:nrow(dg1),12)
dg4=dg1[dg3,]
estimate=predict(selectedMod,newdata=dg4)
estimate
sum.estimate=sum(estimate)
dg5=data.frame(estimate,sum.estimate)

13       34       18       37       38        2       21       43 
417.0408 219.0364 293.3812 367.1180 397.1968 387.0510 269.6912 344.5054 
      20       16       33       15 
333.4693 376.5564 323.6482 386.2197